In [91]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import plotly.express as px
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import euclidean_distances


In [2]:
dados = pd.read_csv("../data/files/dados_reduzidos.csv",
                    sep=",", encoding="utf-8")

dados_generos = pd.read_csv(
    "../data/files/data_by_genres.csv", sep=",", encoding="utf-8")

dados_anos = pd.read_csv(
    "../data/files/data_by_year.csv", sep=",", encoding="utf-8")

In [3]:
dados_generos.head(2)


,mode,genres,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key
0,1,21st century classical,0.979333,0.162883,1.602977e+05,0.071317,0.606834,0.3616,-31.514333,0.040567,75.336500,0.103783,27.833333,6
1,1,432hz,0.494780,0.299333,1.048887e+06,0.450678,0.477762,0.1310,-16.854000,0.076817,120.285667,0.221750,52.500000,5


In [4]:
dados_anos.head(2)


,mode,year,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key
0,1,1921,0.886896,0.418597,260537.166667,0.231815,0.344878,0.20571,-17.048667,0.073662,101.531493,0.379327,0.653333,2
1,1,1922,0.938592,0.482042,165469.746479,0.237815,0.434195,0.24072,-19.275282,0.116655,100.884521,0.535549,0.140845,10


In [5]:
dados.head(2)


,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,artists_song
0,0.0594,1921,0.982,Sergei Rachmaninoff,0.279,831667,0.211,0,4BJqT0PrAfrxzMOxytFOIz,0.878,10,0.665,-20.096,1,"Piano Concerto No. 3 in D Minor, Op. 30: III. ...",4,1921,0.0366,80.954,Sergei Rachmaninoff - Piano Concerto No. 3 in ...
1,0.0394,1921,0.961,KHP Kridhamardawa Karaton Ngayogyakarta Hadini...,0.328,500062,0.166,0,1o6I8BglA6ylDMrIELygv1,0.913,3,0.101,-14.850,1,Gati Bali,5,1921,0.0339,110.339,KHP Kridhamardawa Karaton Ngayogyakarta Hadini...


In [6]:
dados = dados[dados.year >= 2000]
dados["year"].unique()


array([2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010,
       2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020],
      dtype=int64)

In [7]:
dados.shape


(21546, 20)

In [8]:
dados = dados.drop(["explicit", "key", "mode"], axis=1)


In [9]:
dados.shape


(21546, 17)

In [10]:
dados.isnull().sum()


valence             0
year                0
acousticness        0
artists             0
danceability        0
duration_ms         0
energy              0
id                  0
instrumentalness    0
liveness            0
loudness            0
name                0
popularity          0
release_date        0
speechiness         0
tempo               0
artists_song        0
dtype: int64

In [11]:
dados.isna().sum()


valence             0
year                0
acousticness        0
artists             0
danceability        0
duration_ms         0
energy              0
id                  0
instrumentalness    0
liveness            0
loudness            0
name                0
popularity          0
release_date        0
speechiness         0
tempo               0
artists_song        0
dtype: int64

In [12]:
dados_generos.head(2)


,mode,genres,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key
0,1,21st century classical,0.979333,0.162883,1.602977e+05,0.071317,0.606834,0.3616,-31.514333,0.040567,75.336500,0.103783,27.833333,6
1,1,432hz,0.494780,0.299333,1.048887e+06,0.450678,0.477762,0.1310,-16.854000,0.076817,120.285667,0.221750,52.500000,5


In [13]:
dados_generos = dados_generos.drop(["key", "mode"], axis=1)
dados_generos.head(2)


,genres,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity
0,21st century classical,0.979333,0.162883,1.602977e+05,0.071317,0.606834,0.3616,-31.514333,0.040567,75.336500,0.103783,27.833333
1,432hz,0.494780,0.299333,1.048887e+06,0.450678,0.477762,0.1310,-16.854000,0.076817,120.285667,0.221750,52.500000


In [14]:
dados_generos.isna().sum()


genres              0
acousticness        0
danceability        0
duration_ms         0
energy              0
instrumentalness    0
liveness            0
loudness            0
speechiness         0
tempo               0
valence             0
popularity          0
dtype: int64

In [15]:
dados_generos.isnull().sum()


genres              0
acousticness        0
danceability        0
duration_ms         0
energy              0
instrumentalness    0
liveness            0
loudness            0
speechiness         0
tempo               0
valence             0
popularity          0
dtype: int64

In [16]:
dados_anos.head()


,mode,year,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key
0,1,1921,0.886896,0.418597,260537.166667,0.231815,0.344878,0.205710,-17.048667,0.073662,101.531493,0.379327,0.653333,2
1,1,1922,0.938592,0.482042,165469.746479,0.237815,0.434195,0.240720,-19.275282,0.116655,100.884521,0.535549,0.140845,10
2,1,1923,0.957247,0.577341,177942.362162,0.262406,0.371733,0.227462,-14.129211,0.093949,114.010730,0.625492,5.389189,0
3,1,1924,0.940200,0.549894,191046.707627,0.344347,0.581701,0.235219,-14.231343,0.092089,120.689572,0.663725,0.661017,10
4,1,1925,0.962607,0.573863,184986.924460,0.278594,0.418297,0.237668,-14.146414,0.111918,115.521921,0.621929,2.604317,5


In [17]:
dados_anos.year.unique()


array([1921, 1922, 1923, 1924, 1925, 1926, 1927, 1928, 1929, 1930, 1931,
       1932, 1933, 1934, 1935, 1936, 1937, 1938, 1939, 1940, 1941, 1942,
       1943, 1944, 1945, 1946, 1947, 1948, 1949, 1950, 1951, 1952, 1953,
       1954, 1955, 1956, 1957, 1958, 1959, 1960, 1961, 1962, 1963, 1964,
       1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975,
       1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986,
       1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997,
       1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008,
       2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019,
       2020], dtype=int64)

In [18]:
dados_anos = dados_anos[dados_anos.year >= 2000]
dados_anos = dados_anos.drop(["key", "mode"], axis=1)
dados_anos.year.unique()


array([2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010,
       2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020],
      dtype=int64)

In [19]:
dados_anos.reset_index()


,index,year,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity
0,79,2000,0.289323,0.590918,242724.642638,0.625413,0.101168,0.197686,-8.247766,0.089205,118.999323,0.559475,46.684049
1,80,2001,0.286842,0.583318,240307.796010,0.626986,0.107214,0.187026,-8.305095,0.089182,117.765399,0.541479,48.750125
2,81,2002,0.282624,0.576160,239503.283000,0.641270,0.088048,0.193911,-7.686640,0.084308,119.239738,0.542397,48.655500
3,82,2003,0.256471,0.575763,244670.575230,0.660165,0.083049,0.196976,-7.485545,0.093926,120.914622,0.530504,48.626407
4,83,2004,0.280559,0.567680,237378.708037,0.648868,0.077934,0.202199,-7.601655,0.094239,121.290346,0.524489,49.273143
5,84,2005,0.255764,0.572281,237229.588205,0.653209,0.090194,0.190082,-7.466159,0.093334,121.617967,0.532531,50.953333
6,85,2006,0.279986,0.568230,234042.914359,0.650326,0.077701,0.188289,-7.265501,0.085847,121.798615,0.520028,51.313846
7,86,2007,0.254081,0.563414,241049.962564,0.668305,0.072957,0.196127,-7.044536,0.084347,124.087516,0.516794,51.075897
8,87,2008,0.249192,0.579193,240107.315601,0.671461,0.063662,0.198431,-6.843804,0.077356,123.509934,0.527542,50.630179
9,88,2009,0.261929,0.564190,238140.013265,0.670749,0.075872,0.205252,-7.046015,0.085458,123.463808,0.507170,51.440816


In [20]:
fig = px.line(dados_anos, x="year", y="loudness",
              title="Variação do loudness conforme os anos")
fig.show()


In [21]:
import plotly.graph_objects as go


In [22]:
fig = go.Figure()


In [23]:
fig.add_trace(go.Scatter(
    x=dados_anos["year"], y=dados_anos["acousticness"], name="Acousticness"))
fig.add_trace(go.Scatter(
    x=dados_anos["year"], y=dados_anos["valence"], name="Valence"))
fig.add_trace(go.Scatter(
    x=dados_anos["year"], y=dados_anos["danceability"], name="Danceability"))
fig.add_trace(go.Scatter(
    x=dados_anos["year"], y=dados_anos["energy"], name="Energy"))
fig.add_trace(go.Scatter(
    x=dados_anos["year"], y=dados_anos["instrumentalness"], name="Instrumentalness"))
fig.add_trace(go.Scatter(
    x=dados_anos["year"], y=dados_anos["liveness"], name="Liveness"))
fig.add_trace(go.Scatter(
    x=dados_anos["year"], y=dados_anos["speechiness"], name="Speechiness"))
fig.show()


In [24]:
dados_generos.head(2)


,genres,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity
0,21st century classical,0.979333,0.162883,1.602977e+05,0.071317,0.606834,0.3616,-31.514333,0.040567,75.336500,0.103783,27.833333
1,432hz,0.494780,0.299333,1.048887e+06,0.450678,0.477762,0.1310,-16.854000,0.076817,120.285667,0.221750,52.500000


In [25]:
dados_generos["genres"].value_counts().sum()


2973

In [26]:
dados_generos1 = dados_generos.drop('genres', axis=1)
dados_generos1.head(2)


,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity
0,0.979333,0.162883,1.602977e+05,0.071317,0.606834,0.3616,-31.514333,0.040567,75.336500,0.103783,27.833333
1,0.494780,0.299333,1.048887e+06,0.450678,0.477762,0.1310,-16.854000,0.076817,120.285667,0.221750,52.500000


In [27]:
SEED = 1234
np.random.seed(1224)

pca_pipeline = Pipeline([('scaler', StandardScaler()),
                        ('PCA', PCA(n_components=2, random_state=SEED))])

genre_embedding_pca = pca_pipeline.fit_transform(dados_generos1)

projection = pd.DataFrame(columns=['x', 'y'], data=genre_embedding_pca)


In [28]:
print(f"Projection: {projection.shape}")

Projection: (2973, 2)


In [29]:
k_means_pca = KMeans(n_clusters=5, verbose=False, random_state=SEED)
k_means_pca.fit(projection)

dados_generos["cluster_pca"] = k_means_pca.predict(projection)
projection["cluster_pca"] = k_means_pca.predict(projection)


In [30]:
projection["generos"] = dados_generos['genres']
projection.head()


,x,y,cluster_pca,generos
0,5.910268,-0.011146,2,21st century classical
1,2.787093,4.498483,2,432hz
2,-0.757538,-1.258495,0,8-bit
3,1.020521,-0.931690,3,[]
4,0.753911,-0.999861,3,a cappella


In [31]:
fig = px.scatter(projection, x="x", y="y", color="cluster_pca",
                 hover_data=['x', 'y', 'generos'])
fig.show()


In [32]:
# quantidade de radio que são explicadas
pca_pipeline[1].explained_variance_ratio_.sum()


0.4927066994726641

In [33]:
# quantidade de colunas ou clusters que são explicadas
pca_pipeline[1].explained_variance_.sum()


5.421597305805697

<h1>Fazendo Cluster com Músicas</h1>

<h1></h1>

In [45]:
dados.head()

,valence,year,acousticness,artists,danceability,duration_ms,energy,id,instrumentalness,liveness,loudness,name,popularity,release_date,speechiness,tempo,artists_song
12873,0.285,2000,0.00239,Coldplay,0.429,266773,0.661,3AJwUDP919kvQ9QcozQPxg,0.000121,0.2340,-7.227,Yellow,84,2000-07-10,0.0281,173.372,Coldplay - Yellow
12874,0.613,2000,0.14300,OutKast,0.843,270507,0.806,0I3q5fE6wg7LIfHGngUTnV,0.000000,0.0771,-5.946,Ms. Jackson,80,2000-10-31,0.2690,94.948,OutKast - Ms. Jackson
12875,0.400,2000,0.00958,Linkin Park,0.556,216880,0.864,60a0Rd6pjrkxjPbaKzXjfq,0.000000,0.2090,-5.870,In the End,84,2000-10-24,0.0584,105.143,Linkin Park - In the End
12876,0.543,2000,0.00664,3 Doors Down,0.545,233933,0.865,6ZOBP3NvffbU4SZcrnt1k6,0.000011,0.1680,-5.708,Kryptonite,78,2000,0.0286,99.009,3 Doors Down - Kryptonite
12877,0.760,2000,0.03020,Eminem,0.949,284200,0.661,3yfqSUWxFvZELEM4PmlwIR,0.000000,0.0454,-4.244,The Real Slim Shady,80,2000-05-23,0.0572,104.504,Eminem - The Real Slim Shady


In [59]:
music_dumie = pd.concat([dados, pd.get_dummies(
    dados['artists'], prefix='artists')], axis=1).drop(['artists'], axis=1)
music_dumie.head()

,valence,year,acousticness,danceability,duration_ms,energy,id,instrumentalness,liveness,loudness,...,artists_Zedd,artists_alt-J,artists_blackbear,artists_blink-182,artists_deadmau5,artists_for KING & COUNTRY,artists_fun.,artists_girl in red,artists_iann dior,artists_オメガトライブ
12873,0.285,2000,0.00239,0.429,266773,0.661,3AJwUDP919kvQ9QcozQPxg,0.000121,0.2340,-7.227,...,0,0,0,0,0,0,0,0,0,0
12874,0.613,2000,0.14300,0.843,270507,0.806,0I3q5fE6wg7LIfHGngUTnV,0.000000,0.0771,-5.946,...,0,0,0,0,0,0,0,0,0,0
12875,0.400,2000,0.00958,0.556,216880,0.864,60a0Rd6pjrkxjPbaKzXjfq,0.000000,0.2090,-5.870,...,0,0,0,0,0,0,0,0,0,0
12876,0.543,2000,0.00664,0.545,233933,0.865,6ZOBP3NvffbU4SZcrnt1k6,0.000011,0.1680,-5.708,...,0,0,0,0,0,0,0,0,0,0
12877,0.760,2000,0.03020,0.949,284200,0.661,3yfqSUWxFvZELEM4PmlwIR,0.000000,0.0454,-4.244,...,0,0,0,0,0,0,0,0,0,0


In [47]:
print(f"dados ==> {dados.shape}")
print(f'dados dummies ==> {music_dumie.shape}')


dados ==> (21546, 17)
dados dummies ==> (21546, 1579)


In [60]:
pca_pipeline = Pipeline([('scaler', StandardScaler()),
                        ('PCA', PCA(n_components=0.7, random_state=SEED))])

music_embedding_pca = pca_pipeline.fit_transform(music_dumie.drop(
    ['id', 'name', 'artists_song', 'release_date'], axis=1))

projection_music = pd.DataFrame(data=music_embedding_pca)

In [49]:
projection_music.head()

,0,1,2,3,4,5,6,7,8,9,...,1084,1085,1086,1087,1088,1089,1090,1091,1092,1093
0,0.070576,0.520117,2.216909,-0.910164,0.528643,-0.154408,-0.298385,-0.564471,-2.690171,1.975150,...,9.141838e-13,4.231086e-13,1.227733e-12,-2.762468e-12,3.138652e-13,-4.141317e-13,-6.126364e-13,1.680376e-12,2.988916e-13,-1.238882e-12
1,-1.434962,0.045598,-1.674609,1.906482,0.381099,-1.111431,-1.133168,0.506212,-1.713906,0.350026,...,5.599696e-12,-1.547467e-13,-1.782485e-12,3.446030e-12,-2.167982e-12,-4.096552e-12,-2.559774e-12,-5.761776e-13,-4.565956e-13,3.232006e-12
2,-1.029663,0.529666,0.804739,-0.376150,-0.200566,-0.213421,0.789295,1.980178,-1.639476,1.264259,...,-7.108716e-14,-4.166861e-14,-3.609884e-13,7.655617e-14,9.835362e-13,-2.077100e-12,-3.399976e-15,1.519147e-13,1.995163e-13,5.209912e-13
3,-0.964129,1.169663,0.447230,-1.087722,0.349689,-0.573343,-0.590508,0.942237,-1.313010,0.962899,...,4.259341e-12,-1.332094e-12,1.765688e-12,-1.355705e-12,1.688033e-12,-1.029018e-11,-4.973098e-12,-1.959781e-12,3.310829e-12,7.360482e-13
4,-1.761283,-0.381690,-1.215170,1.338292,1.387351,-1.349616,0.062583,0.458048,-2.160498,0.497874,...,1.447069e-12,-7.357915e-13,-8.044237e-13,1.914923e-13,-3.217762e-13,-1.457312e-12,-2.192139e-13,3.004473e-14,-1.511527e-13,1.313718e-12


In [50]:
print(f'components embdding ==> {pca_pipeline[1].n_components_}')
print(f'projection music ==> {projection_music.shape[1]}')


components embdding ==> 1094
projection music ==> 1094


In [61]:
# Aplicando cluster com k-means
k_means_pca_pipeline = KMeans(n_clusters=50, verbose=False, random_state=SEED)
k_means_pca_pipeline.fit(projection_music)

dados["cluster_pca"] = k_means_pca_pipeline.predict(projection_music)
projection_music["cluster_pca"] = k_means_pca_pipeline.predict(projection_music)


In [101]:
projection_music['artist'] = dados['artists'].tolist()
projection_music['song'] = dados['artists_song'].tolist()
projection_music['id'] = dados['id'].tolist()


In [102]:
projection_music.head()


,0,1,2,3,4,5,6,7,8,9,...,1087,1088,1089,1090,1091,1092,cluster_pca,artist,song,id
0,-0.241154,0.733875,1.956476,-1.101446,-1.429677,0.199257,-0.294232,-0.271743,-0.651590,-2.972126,...,-1.771543e-12,-1.527682e-12,3.570560e-12,-6.813581e-13,2.448648e-12,-4.301139e-12,10,Coldplay,Coldplay - Yellow,3AJwUDP919kvQ9QcozQPxg
1,-1.662018,0.168516,-1.684723,1.907392,-0.302240,0.440816,-1.145794,-1.011177,0.653580,-1.729786,...,-2.588265e-12,-4.686676e-12,-2.173602e-12,-2.375629e-12,-3.712363e-12,8.297544e-13,10,OutKast,OutKast - Ms. Jackson,0I3q5fE6wg7LIfHGngUTnV
2,-1.275374,0.586599,0.715808,-0.453987,-0.593384,-0.368734,-0.404646,1.146053,1.757059,-1.707491,...,9.880483e-13,2.263894e-13,1.733650e-12,1.298813e-12,-5.438132e-13,-2.594719e-12,10,Linkin Park,Linkin Park - In the End,60a0Rd6pjrkxjPbaKzXjfq
3,-1.269886,1.232251,0.424326,-1.134686,-0.404174,0.253390,-0.665770,-0.388329,0.951961,-1.373104,...,-4.980779e-12,-6.063962e-12,7.792205e-12,-2.885102e-12,-3.799492e-12,-4.069629e-12,10,3 Doors Down,3 Doors Down - Kryptonite,6ZOBP3NvffbU4SZcrnt1k6
4,-1.949230,-0.286887,-1.295292,1.302285,-0.594234,1.471436,-1.176993,0.141526,0.461812,-2.224429,...,-7.247057e-13,-2.148874e-12,-1.098721e-13,2.411327e-13,3.301265e-13,-4.308480e-13,10,Eminem,Eminem - The Real Slim Shady,3yfqSUWxFvZELEM4PmlwIR


In [78]:
fig = px.scatter(projection_music, x=0, y=1, color="cluster_pca",
                 hover_data=[0, 1, 'song'])
fig.show()


<h1>Sistema de Recomendação</h1>

In [103]:
nome_musica = "Ed Sheeran - Shape of You"

In [86]:
# find cluster
cluster_found = list(projection_music[projection_music['song'] == nome_musica]['cluster_pca'])[0]

10

In [108]:
# music recommended
music_recommended = projection_music[projection_music['cluster_pca'] == cluster_found][[0, 1, 'id','song', 'artist']]
music_recommended.head()

,0,1,id,song,artist
0,-0.241154,0.733875,3AJwUDP919kvQ9QcozQPxg,Coldplay - Yellow,Coldplay
1,-1.662018,0.168516,0I3q5fE6wg7LIfHGngUTnV,OutKast - Ms. Jackson,OutKast
2,-1.275374,0.586599,60a0Rd6pjrkxjPbaKzXjfq,Linkin Park - In the End,Linkin Park
3,-1.269886,1.232251,6ZOBP3NvffbU4SZcrnt1k6,3 Doors Down - Kryptonite,3 Doors Down
4,-1.949230,-0.286887,3yfqSUWxFvZELEM4PmlwIR,Eminem - The Real Slim Shady,Eminem


In [109]:
x_musica = list(projection_music[projection_music['song'] ==nome_musica][0])[0]
y_musica = list(projection_music[projection_music['song'] ==nome_musica][1])[0]

In [110]:
distances = euclidean_distances(music_recommended[[0, 1]], [[x_musica, y_musica]])
music_recommended['distances'] = distances
recommended = music_recommended.sort_values('distances').head(10)
recommended


,0,1,id,song,artist,distances
2480,-0.714837,-2.893703,7qiZfU4dY1lWllzX7mPBI3,Ed Sheeran - Shape of You,Ed Sheeran,0.000000
21316,-0.691558,-2.889445,6Gg25EZRbQd4IHiJz2KSY0,Kodak Black - Testimony,Kodak Black,0.023665
2530,-0.740928,-2.878681,343YBumqHu19cGoGARUTsd,Drake - Fake Love,Drake,0.030106
18069,-0.711734,-2.850371,26rIWI1QTPClaOOI4Kbxhe,KYLE - What It Is (feat. Chris Brown),KYLE,0.043443
5290,-0.748603,-2.860372,3qN5qMTKyEEmiTZD38BNTT,Drake - I'm Upset,Drake,0.047445
2953,-0.721370,-2.943253,6KfoDhO4XUWSbnyKjNp9c4,Conan Gray - Maniac,Conan Gray,0.049979
2590,-0.655654,-2.888650,2IRZnDFmlqMuOrYOLnZZyc,Meek Mill - Going Bad (feat. Drake),Meek Mill,0.059398
12161,-0.660235,-2.869692,7BxpObs6K7Yio5lVb3sMYX,Lil Mosey - G Walk (with Chris Brown),Lil Mosey,0.059648
2625,-0.658097,-2.870368,6rz0dTA0PdhXImFV5EjM0w,Juice WRLD - Hide (feat. Seezyn),Juice WRLD,0.061350
19635,-0.762792,-2.944855,3frUvGrmGcay91lvFbOgsN,Kygo - Like It Is,Kygo,0.070116
